In [76]:
from datetime import datetime
import numpy as np
from numba import cuda
from tabulate import tabulate
import matplotlib.pyplot as plt
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32 

In [77]:
TPB = 16
count_iter = 16
BPG = int(count_iter/TPB)

In [78]:
def Calcul_on_CPU(count_iter, N):
    res = np.zeros(count_iter)
    for i in range(count_iter):
        x = np.random.uniform(size=N)
        y = np.random.uniform(size=N)
        z = x ** 2 + y ** 2 <= 1
        res[i] = 4.0 * sum(z) / N
    return res

In [79]:
@cuda.jit 
def pi_calcul(res, rng):
  i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  h = 0
  if i < len(rng):
    x = xoroshiro128p_uniform_float32(rng, i)
    y = xoroshiro128p_uniform_float32(rng, i)
    if x ** 2 + y ** 2 <= 1.0:
      h += 1
  cuda.atomic.add(res, 0, h)

In [80]:
def Calcul_on_GPU(N):

  rng_states = create_xoroshiro128p_states(N, seed=1)
  res = [0]

  dev_res = cuda.to_device(res)
  pi_calcul[N, TPB](dev_res, rng_states)
  tmp_ = dev_res.copy_to_host()
  pi_ = 4 * tmp_[0] / N
  return pi_
  

In [81]:
def main(iter_ , N):
  #Эксперименты на CPU и GPU
  for i in range(iter_):
    print("Номер итерации:" + str(i+1))
    start = datetime.now()
    cpu_pi = Calcul_on_CPU(count_iter, N).mean()
    print("Время выполения на CPU - "+ str(datetime.now()- start))
    print("Число Pi - " + str(cpu_pi))
    start = datetime.now()
    gpu_pi = Calcul_on_GPU(N)
    print("Время выполения на GPU - "+ str(datetime.now()- start))
    print("Число Pi - " + str(gpu_pi))
    print("")
    N = N + 10000
  return None



In [83]:
main(10, 10000)

Номер итерации:1
Время выполения на CPU - 0:00:00.449358
Число Pi - 3.1443250000000003
Время выполения на GPU - 0:00:00.005631
Число Pi - 3.1552

Номер итерации:2
Время выполения на CPU - 0:00:00.920117
Число Pi - 3.141425
Время выполения на GPU - 0:00:00.008749
Число Pi - 3.1578

Номер итерации:3
Время выполения на CPU - 0:00:01.329403
Число Pi - 3.1395999999999997
Время выполения на GPU - 0:00:00.011196
Число Pi - 3.1508

Номер итерации:4
Время выполения на CPU - 0:00:01.755199
Число Pi - 3.1411187500000004
Время выполения на GPU - 0:00:00.014902
Число Pi - 3.1471

Номер итерации:5
Время выполения на CPU - 0:00:02.278638
Число Pi - 3.1429650000000002
Время выполения на GPU - 0:00:00.019343
Число Pi - 3.14304

Номер итерации:6
Время выполения на CPU - 0:00:02.695535
Число Pi - 3.1409791666666664
Время выполения на GPU - 0:00:00.020945
Число Pi - 3.1470666666666665

Номер итерации:7
Время выполения на CPU - 0:00:03.095004
Число Pi - 3.1428964285714285
Время выполения на GPU - 0:00:00.0